In [2]:
import os, gc
import pandas as pd
import numpy as np
from sklearn.metrics import precision_score, recall_score, f1_score
from sklearn.utils.class_weight import compute_class_weight
from concurrent.futures import ThreadPoolExecutor, as_completed

from google.colab import drive

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
from torch.optim.lr_scheduler import ReduceLROnPlateau

drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# Models

In [9]:
class Attention(nn.Module):
    def __init__(self, hidden_dim, num_heads):
        super(Attention, self).__init__()
        self.attention = nn.MultiheadAttention(embed_dim=hidden_dim, num_heads=num_heads, batch_first=True)

    def forward(self, x):
        output, attention_weights = self.attention(x, x, x)
        return output, attention_weights

class LSTMModule(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, dropout_prob, device):
        super(LSTMModule, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.device = device

        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)
        self.attention = Attention(hidden_size, 4)
        self.dropout = nn.Dropout(dropout_prob)

    def forward(self, x):
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(self.device)
        c0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(self.device)

        out, _ = self.lstm(x, (h0, c0))
        out, attention_weights = self.attention(out)

        out = torch.mean(out, dim=1)

        out = self.dropout(out)

        return out, attention_weights

class CNNModule(nn.Module):
    def __init__(self, input_channels, dropout):
        super(CNNModule, self).__init__()

        self.conv1 = nn.Conv1d(input_channels, 64, kernel_size=3, padding=1)
        self.conv2 = nn.Conv1d(64, 128, kernel_size=3, padding=1)
        self.conv3 = nn.Conv1d(128, 256, kernel_size=3, padding=1)

        self.pool1 = nn.MaxPool1d(kernel_size=2, stride=2)
        self.pool2 = nn.MaxPool1d(kernel_size=2, stride=2)
        self.pool3 = nn.MaxPool1d(kernel_size=2, stride=2)

        self.dropout = nn.Dropout(p=dropout)

        self.relu1 = nn.ReLU()
        self.relu2 = nn.ReLU()
        self.relu3 = nn.ReLU()

        self.attention = Attention(256, 4)
        self.global_avg_pool = nn.AdaptiveAvgPool1d(1)

    def forward(self, x):
        x = x.permute(0, 2, 1)

        x = self.relu1(self.conv1(x))
        # print('Shape before pool', 1, x.shape)
        x = self.pool1(x)
        x = self.dropout(x)

        x = self.relu2(self.conv2(x))
        # print('Shape before pool', 2, x.shape)
        x = self.pool2(x)
        x = self.dropout(x)

        x = self.relu3(self.conv3(x))
        # print('Shape before pool', 3, x.shape)
        x = self.pool3(x)
        x = self.dropout(x)

        x = x.permute(0, 2, 1)
        x, attention_weights = self.attention(x)

        x = x.permute(0, 2, 1)
        x = self.global_avg_pool(x)
        x = x.squeeze(-1)

        return x, attention_weights

class CNNModel(nn.Module):
    def __init__(self, input_channels, output_size, dropout):
        super(CNNModel, self).__init__()

        self.cnn_module = CNNModule(input_channels, dropout)

        self.fc = nn.Sequential(
            nn.Linear(256, 128),
            nn.ReLU(),
            nn.Dropout(dropout),
            nn.Linear(128, 64),
            nn.ReLU(),
            nn.Dropout(dropout),
            nn.Linear(64, output_size),
        )

    def forward(self, x):
        x, attention_weights = self.cnn_module(x)
        x = self.fc(x)
        return x, attention_weights

class LSTMModel(nn.Module):
    def __init__(self, input_size, hidden_size, output_size, device, dropout_prob, num_layers=1):
        super(LSTMModel, self).__init__()

        self.lstm_module = LSTMModule(input_size, hidden_size, num_layers, dropout_prob, device)

        self.fc = nn.Sequential(
            nn.Linear(hidden_size, 128),
            nn.ReLU(),
            nn.Dropout(dropout_prob),
            nn.Linear(128, 64),
            nn.ReLU(),
            nn.Dropout(dropout_prob),
            nn.Linear(64, output_size),
        )

    def forward(self, x):
        x, attention_weights = self.lstm_module(x)
        x = self.fc(x)
        return x, attention_weights

class MultimodalModel(nn.Module):
    def __init__(self, ip_size_landmarks, ip_size_aus, ip_size_gaze, hidden_size, output_size, device, dropout_prob, num_layers=1):
        super(MultimodalModel, self).__init__()

        self.landmarks_lstm = LSTMModule(ip_size_landmarks, hidden_size, num_layers, dropout_prob, device)

        self.aus_cnn = CNNModule(ip_size_aus, dropout_prob)
        self.gaze_cnn = CNNModule(ip_size_gaze, dropout_prob)

        self.combined_attention = Attention(hidden_size + 256 * 2, num_heads=4)

        self.fc = nn.Sequential(
            nn.Linear(hidden_size + 256 * 2, 128),
            nn.ReLU(),
            nn.Dropout(dropout_prob),
            nn.Linear(128, 64),
            nn.ReLU(),
            nn.Dropout(dropout_prob),
            nn.Linear(64, output_size),
        )

    def forward(self, landmarks, aus, gaze):
        out_landmarks, attn_weights_landmarks = self.landmarks_lstm(landmarks)
        
        out_aus, attn_weights_aus = self.aus_cnn(aus)
        out_gaze, attn_weights_gaze = self.gaze_cnn(gaze)

        combined_out = torch.cat((out_landmarks, out_aus, out_gaze), dim=1).unsqueeze(1)

        if self.combined_attn:
            combined_out, combined_attention_weights = self.combined_attention(combined_out)

        combined_out = combined_out.squeeze(1)

        out = self.fc(combined_out)
        
        if self.combined_attn:
            return out, (attn_weights_landmarks, attn_weights_aus, attn_weights_gaze, combined_attention_weights)
        else:
            return out, (attn_weights_landmarks, attn_weights_aus, attn_weights_gaze)


# Custom Dataset

In [3]:
class DAICDataset(Dataset):

    def __init__(self, split_details, chunk_size, is_train=True, max_workers=4):
        self.split_details = pd.read_csv(split_details)
        self.is_train = is_train
        self.chunk_size = chunk_size
        self.base_url = '/content/drive/MyDrive/Dissertation/data/data/'
        self.max_workers = max_workers

        self.data_cache = {}
        self.chunk_info = self._calculate_chunks_parallel()

    def __len__(self):
        return sum(num_chunks for _, num_chunks in self.chunk_info)

    def _load_data(self, pid):
        base_path = os.path.join(self.base_url, str(pid))
        landmarks = pd.read_csv(f'{base_path}/{pid}_CLNF_features.csv')
        aus = pd.read_csv(f'{base_path}/{pid}_CLNF_AUs.csv')
        gaze = pd.read_csv(f'{base_path}/{pid}_CLNF_gaze.csv')

        valid_indices = landmarks.index.intersection(aus.index).intersection(gaze.index)

        landmarks = landmarks.loc[valid_indices].iloc[:, 4:].values.astype(np.float32)
        aus = aus.loc[valid_indices].iloc[:, 4:].values.astype(np.float32)
        gaze = gaze.loc[valid_indices].iloc[:, 4:].values.astype(np.float32)

        return {
            'landmarks': landmarks,
            'aus': aus,
            'gaze': gaze,
            'num_frames': len(landmarks)
        }

    def _calculate_chunks_parallel(self):
        chunk_info = []

        # parallel load data
        with ThreadPoolExecutor(max_workers=self.max_workers) as executor:
            futures = {executor.submit(self._load_data, int(row['Participant_ID'])): row['Participant_ID']
                       for _, row in self.split_details.iterrows()}

            for future in as_completed(futures):
                pid = futures[future]
                data = future.result()
                self.data_cache[pid] = data
                num_chunks = (data['num_frames'] + self.chunk_size - 1) // self.chunk_size
                chunk_info.append((pid, num_chunks))

        return chunk_info

    def _load_chunk_data(self, pid, chunk_idx):
        data = self.data_cache[pid]
        start = chunk_idx * self.chunk_size
        end = min(start + self.chunk_size, data['num_frames'])

        landmarks = data['landmarks'][start:end]
        aus = data['aus'][start:end]
        gaze = data['gaze'][start:end]

        # pad data
        if end - start < self.chunk_size:
          padding_size = self.chunk_size - (end - start)
          landmarks = np.pad(landmarks, ((0, padding_size), (0, 0)), mode='constant')
          aus = np.pad(aus, ((0, padding_size), (0, 0)), mode='constant')
          gaze = np.pad(gaze, ((0, padding_size), (0, 0)), mode='constant')

        sample = {
            'pid': pid,
            'landmarks': torch.tensor(landmarks, dtype=torch.float32),
            'aus': torch.tensor(aus, dtype=torch.float32),
            'gaze': torch.tensor(gaze, dtype=torch.float32)
        }

        if self.is_train:
            label = self.split_details[self.split_details['Participant_ID'] == pid]['PHQ8_Binary'].values[0]
            sample["label"] = label
        else:
            actual = self.split_details[self.split_details['Participant_ID'] == pid]['PHQ_Binary'].values[0]
            sample["actual"] = actual

        return sample

    def __getitem__(self, idx):
        cumulative_chunks = 0
        for pid, num_chunks in self.chunk_info:
            if idx < cumulative_chunks + num_chunks:
                chunk_idx = idx - cumulative_chunks
                return self._load_chunk_data(pid, chunk_idx)
            cumulative_chunks += num_chunks

        raise IndexError("Index out of range")


# Training function

In [4]:
def train_model(config, train_dataset, val_dataset, class_weights):
    feature_size = {"landmarks": 136, "aus": 20, "gaze": 12}
    method = "chunk"
    m_name = f"{config['model']}_{config['feature']}_{method}"

    base_url = '/content/drive/MyDrive/Dissertation/data/'
    write_url = '/content/drive/MyDrive/Dissertation/models/'

    # initialise model
    device = "cuda" if torch.cuda.is_available() else "cpu"
    if config['model'] == "lstm":
        model = LSTMModel(input_size=feature_size[config['feature']],
                          hidden_size=config['hidden_size'],
                          dropout_prob=config['dropout'],
                          num_layers=config['lstm_layer'],
                          output_size=1,
                          device=device)
    elif config['model'] == "cnn":
        model = CNNModel(input_channels=feature_size[config['feature']],
                         output_size=1,
                         dropout=config['dropout'])
    elif config['model'] == "multimodal":
        model = MultimodalModel(ip_size_landmarks=feature_size['landmarks'],
                                ip_size_aus=feature_size['aus'],
                                ip_size_gaze=feature_size['gaze'],
                                hidden_size=config['hidden_size'],
                                dropout_prob=config['dropout'],
                                output_size=1,
                                device=device)
    model.to(device)

    class_weights = torch.tensor(class_weights, dtype=torch.float).to(device)

    train_loader = DataLoader(train_dataset, batch_size=config['batch_size'], shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=config['batch_size'], shuffle=True)

    optimizer = optim.Adam(model.parameters(), lr=config['lr'], weight_decay=1e-5)
    criterion = nn.BCEWithLogitsLoss(pos_weight=class_weights[1])

    t_precisions, t_recalls ,t_f1s, t_losses = [], [], [], []
    v_precisions, v_recalls ,v_f1s, v_losses = [], [], [], []
    best_thresholds = []
    best_f1 = 0.0
    best_val_loss = float('inf')

    best_threshold = 0.5
    thresholds = np.arange(0.3, 0.7, 0.01)

    patience = 10
    noimp_count = 0

    for epoch in range(config['epochs']):
        model.train()
        torch.cuda.empty_cache()
        gc.collect()
        print(f'Epoch: {epoch}')

        epoch_loss = 0.0
        all_labels_train, all_preds_train = [], []

        # Training loop
        for batch in train_loader:
            optimizer.zero_grad()

            if config['model'] == "multimodal":
                landmarks, aus, gaze = batch['landmarks'].to(device), batch['aus'].to(device), batch['gaze'].to(device)
                label = batch['label'].to(device).float()
                output, attention_weights = model(landmarks, aus, gaze)
            else:
                data = batch[config['feature']].to(device)
                label = batch['label'].to(device).float()
                output, attention_weights = model(data)

            # print(output)
            output = output.squeeze(-1)

            # print(output, label)
            loss = criterion(output, label)
            loss.backward()
            optimizer.step()

            epoch_loss += loss.item()

            outputs = torch.sigmoid(output)
            all_labels_train.extend(label.cpu().numpy())
            all_preds_train.extend(outputs.detach().cpu().numpy())

        avg_train_loss = epoch_loss / len(train_loader)
        t_losses.append(avg_train_loss)

        best_epoch_train_f1 = 0.0
        best_epoch_train_recall = 0.0
        best_epoch_train_precision = 0.0

        best_t_train = 0.0

        # find best threshold
        for threshold in thresholds:
            predicted = (all_preds_train > threshold).astype(float)
            train_precision = precision_score(all_labels_train, predicted, zero_division=0)
            train_recall = recall_score(all_labels_train, predicted)
            train_f1 = f1_score(all_labels_train, predicted)

            if train_f1 > best_epoch_train_f1:
                best_epoch_train_f1 = train_f1
                best_epoch_train_recall = train_recall
                best_epoch_train_precision = train_precision
                best_t_train = threshold

        t_f1s.append(best_epoch_train_f1)
        t_precisions.append(best_epoch_train_precision)
        t_recalls.append(best_epoch_train_recall)

        print(f'Training Loss: {avg_train_loss:.4f}, Best Precision: {best_epoch_train_precision:.4f}, Recall: {best_epoch_train_recall:.4f}, F1-Score: {best_epoch_train_f1:.4f}')
        print(f"Best Train Threshold: {best_t_train}")

        # Evaluation loop
        model.eval()
        val_loss = 0.0
        all_labels_val = []
        all_preds_val = []

        with torch.no_grad():
            for batch in val_loader:
                if config['model'] == "multimodal":
                    landmarks, aus, gaze = batch['landmarks'].to(device), batch['aus'].to(device), batch['gaze'].to(device)
                    label = batch['label'].to(device).float()
                    output, attention_weights = model(landmarks, aus, gaze)
                else:
                    data = batch[config['feature']].to(device)
                    label = batch['label'].to(device).float()
                    output, attention_weights = model(data)

                output = output.squeeze(-1)

                val_loss += criterion(output, label).item()
                outputs = torch.sigmoid(output)
                all_labels_val.extend(label.cpu().numpy())
                all_preds_val.extend(outputs.detach().cpu().numpy())

        avg_val_loss = val_loss / len(val_loader)
        v_losses.append(avg_val_loss)

        best_epoch_val_f1 = 0.0
        best_epoch_val_recall = 0.0
        best_epoch_val_precision = 0.0

        # get best threshold
        for threshold in thresholds:
            predicted = (all_preds_val > threshold).astype(float)
            val_precision = precision_score(all_labels_val, predicted, zero_division=0)
            val_recall = recall_score(all_labels_val, predicted)
            val_f1 = f1_score(all_labels_val, predicted)

            if val_f1 > best_epoch_val_f1:
                best_epoch_val_f1 = val_f1
                best_epoch_val_recall = val_recall
                best_epoch_val_precision = val_precision
                best_threshold = threshold

        v_f1s.append(best_epoch_val_f1)
        v_precisions.append(best_epoch_val_precision)
        v_recalls.append(best_epoch_val_recall)

        best_thresholds.append(best_threshold)

        print(f'Validation Loss: {avg_val_loss:.4f}, Best Precision: {best_epoch_val_precision:.4f}, Recall: {best_epoch_val_recall:.4f}, F1-Score: {best_epoch_val_f1:.4f}')
        print(f'Best threshold: {best_threshold}')
        print()

        data = {'model': model,
                'optimizer': optimizer,
                'best_threshold': best_threshold}


        if (best_epoch_val_f1 > best_f1) or (best_epoch_val_f1 == best_f1 and avg_val_loss < best_val_loss):
            best_f1 = best_epoch_val_f1
            best_val_loss = avg_val_loss
            noimp_count = 0
            print(f'Best validation Loss: {avg_val_loss:.4f}, F1: {best_f1:.4f}')
            _save_model(m_name, epoch, data, best_model=True)
        else:
            noimp_count += 1

        # # uncomment if you want early stopping
        # if noimp_count >= patience:
        #     print(f"Early stopping triggered after {epoch} epochs.")
        #     _save_model(m_name, epoch, data)
        #     break

        if epoch != 0 and ((epoch % 10 == 0) or (epoch == config['epochs'] - 1)):
            _save_model(m_name, epoch, data)

    pd.DataFrame({
        'train_loss': t_losses,
        'train_precision': t_precisions,
        'train_recall': t_recalls,
        'train_f1': t_f1s,
        'val_loss': v_losses,
        'val_precision': v_precisions,
        'val_recall': v_recalls,
        'val_f1': v_f1s,
        'best_threshold': best_thresholds
    }).to_csv(f"{write_url}{m_name}/progress.csv", index=False)

def _save_model(m_name, epoch, data, best_model=False):
    write_url = '/content/drive/MyDrive/Dissertation/models/'

    if best_model:
        dir = os.path.join(write_url, m_name, "best_model")
    else:
        dir = os.path.join(write_url, m_name, str(epoch))

    os.makedirs(dir, exist_ok=True)

    torch.save(data['model'].state_dict(), os.path.join(dir, "model.pth"))
    torch.save(data['optimizer'].state_dict(), os.path.join(dir, "optim.pth"))
    torch.save(data['scheduler'].state_dict(), os.path.join(dir, "lrscheduler.pth"))

# Testing function

In [5]:
def test_model(config, test_dataset, class_weights):
    feature_size = {"landmarks": 136, "aus": 20, "gaze": 12}
    method = "chunk"
    m_name = f"{config['model']}_{config['feature']}_{method}"

    base_path = '/content/drive/MyDrive/Dissertation/models/'
    model_path = f'{base_path}{m_name}/best_model/model.pth'
    progress_path = f'{base_path}{m_name}/progress.csv'

    # model loading and initialisation
    device = "cuda" if torch.cuda.is_available() else "cpu"
    if config['model'] == "lstm":
        model = LSTMModel(input_size=feature_size[config['feature']],
                          hidden_size=config['hidden_size'],
                          dropout_prob=config['dropout'],
                          output_size=1,
                          num_layers=config['lstm_layer'],
                          device=device)
    elif config['model'] == "cnn":
        model = CNNModel(input_channels=feature_size[config['feature']],
                         output_size=1,
                         dropout=config['dropout'])
    elif config['model'] == "multimodal":
        model = MultimodalModel(ip_size_landmarks=feature_size['landmarks'],
                                ip_size_aus=feature_size['aus'],
                                ip_size_gaze=feature_size['gaze'],
                                hidden_size=config['hidden_size'],
                                dropout_prob=config['dropout'],
                                output_size=1,
                                device=device)
    model.to(device)

    model.load_state_dict(torch.load(model_path, map_location=device))

    # get best threshold from progress.csv file
    progress_df = pd.read_csv(progress_path)

    best_model_row = None
    best_f1 = 0.0
    best_val_loss = float('inf')

    for index, row in progress_df.iterrows():
        epoch_val_f1 = row['val_f1']
        avg_val_loss = row['val_loss']

        if (epoch_val_f1 > best_f1) or (epoch_val_f1 == best_f1 and avg_val_loss < best_val_loss):
            best_f1 = epoch_val_f1
            best_val_loss = avg_val_loss
            best_model_row = row

    best_threshold = best_model_row['best_threshold']

    class_weights = torch.tensor(class_weights, dtype=torch.float).to(device)
    criterion = nn.BCEWithLogitsLoss(pos_weight=class_weights[1])

    test_loader = DataLoader(test_dataset, batch_size=config['batch_size'], shuffle=True)

    model.eval()
    test_loss = 0.0
    all_preds, all_actuals = [], []

    with torch.no_grad():
        for batch in test_loader:
            if config['model'] == "multimodal":
                landmarks, aus, gaze = batch['landmarks'].to(device), batch['aus'].to(device), batch['gaze'].to(device)
                actual = batch['actual'].to(device).float()
                output, _ = model(landmarks, aus, gaze)
            else:
                data = batch[config['feature']].to(device)
                actual = batch['actual'].to(device).float()
                output, _ = model(data)

            output = output.squeeze(-1)
            test_loss += criterion(output, actual).item()

            output = torch.sigmoid(output)
            pred = (output > best_threshold).float()

            all_preds.extend(pred.detach().cpu().numpy())
            all_actuals.extend(actual.cpu().numpy())

    precision = precision_score(all_actuals, all_preds)
    recall = recall_score(all_actuals, all_preds)
    f1 = f1_score(all_actuals, all_preds)

    avg_loss = test_loss / len(test_loader)

    print(f'Test Loss: {avg_loss:.4f}')
    print(f'Precision: {precision:.4f}')
    print(f'Recall: {recall:.4f}')
    print(f'F1-Score: {f1:.4f}')

    result = {
        'model_comb': m_name,
        'test_loss': avg_loss,
        'precision': precision,
        'recall': recall,
        'f1_score': f1
    }

    return result

# Results

### Initialise variable

In [6]:
models = ['multimodal'] #, 'lstm', 'cnn', 'multimodal'
features = ['landmarks', 'aus', 'gaze']

config = {
    "model": "lstm",
    "feature": "landmarks",
    "dropout": 0.2,
    "lr": 0.001,
    "batch_size": 8,
    "hidden_size": 32,
    "epochs": 30,
    "chunk_size": 500,
    "lstm_layer": 3
}

### Load data

In [7]:
base_url = '/content/drive/MyDrive/Dissertation/data'

train_labels_dir = f"{base_url}/train_split_Depression_AVEC2017.csv"
val_labels_dir = f"{base_url}/dev_split_Depression_AVEC2017.csv"

train_dataset = DAICDataset(train_labels_dir, config['chunk_size'], is_train=True)
val_dataset = DAICDataset(val_labels_dir, config['chunk_size'], is_train=True)

train_labels = pd.read_csv(f"{base_url}/train_split_Depression_AVEC2017.csv")['PHQ8_Binary'].values
# print(np.unique(train_labels))
unique, counts = np.unique(train_labels, return_counts=True)

value_counts = dict(zip(unique, counts))

print("Value Counts:\n", value_counts)

class_weights = compute_class_weight(class_weight="balanced", classes=np.unique(train_labels), y=train_labels)
print(class_weights)

Value Counts:
 {0: 77, 1: 30}
[0.69480519 1.78333333]


### Training

In [10]:
for model in models:
    config['model'] = model
    if model == 'multimodal':
        config['feature'] = 'all'
        train_model(config, train_dataset, val_dataset, class_weights)
    else:
        for feature in features:
            print(model, feature)
            config['feature'] = feature
            train_model(config, train_dataset, val_dataset, class_weights)

# train_model(config)

Epoch: 0
Training Loss: 0.8329, Best Precision: 0.2803, Recall: 0.9634, F1-Score: 0.4342
Best Train Threshold: 0.3
Validation Loss: 0.9299, Best Precision: 0.3856, Recall: 1.0000, F1-Score: 0.5565
Best threshold: 0.4100000000000001

Best validation Loss: 0.9299, F1: 0.5565
Epoch: 1
Training Loss: 0.8225, Best Precision: 0.2828, Recall: 0.9686, F1-Score: 0.4377
Best Train Threshold: 0.3
Validation Loss: 0.9028, Best Precision: 0.3865, Recall: 1.0000, F1-Score: 0.5575
Best threshold: 0.35000000000000003

Best validation Loss: 0.9028, F1: 0.5575
Epoch: 2
Training Loss: 0.8046, Best Precision: 0.3131, Recall: 0.8217, F1-Score: 0.4535
Best Train Threshold: 0.38000000000000006
Validation Loss: 1.0248, Best Precision: 0.4032, Recall: 0.7646, F1-Score: 0.5280
Best threshold: 0.31

Epoch: 3
Training Loss: 0.7119, Best Precision: 0.4604, Recall: 0.6748, F1-Score: 0.5473
Best Train Threshold: 0.4100000000000001
Validation Loss: 1.8186, Best Precision: 0.3259, Recall: 0.4633, F1-Score: 0.3826
Best

### Testing

In [11]:
test_labels_dir = "/content/drive/MyDrive/Dissertation/data/full_test_split.csv"
test_dataset = DAICDataset(test_labels_dir, config['chunk_size'], is_train=False)
test_loader = DataLoader(test_dataset, batch_size=config['batch_size'], shuffle=True)

test_labels = pd.read_csv(test_labels_dir)['PHQ_Binary'].values
class_weights = compute_class_weight(class_weight="balanced", classes=np.unique(test_labels), y=test_labels)

In [12]:
# test(config)
all_results = []

for model in models:
    config['model'] = model
    if model == 'multimodal':
        config['feature'] = 'all'
        result = test_model(config, test_dataset, class_weights)
        all_results.append(result)
    else:
      for feature in features:
          print(model, feature)
          config['feature'] = feature

          result = test_model(config, test_dataset, class_weights)
          all_results.append(result)

# Save the results
results_df = pd.DataFrame(all_results)
csv_output_path = '/content/drive/MyDrive/Dissertation/models/test_results.csv'

if os.path.exists(csv_output_path):
    existing_df = pd.read_csv(csv_output_path)
    combined_df = pd.concat([existing_df, results_df], ignore_index=True)
    combined_df.to_csv(csv_output_path, index=False)
else:
    results_df.to_csv(csv_output_path, index=False)

Test Loss: 0.8445
Precision: 0.3133
Recall: 0.9941
F1-Score: 0.4764
